In [1]:
import os
import json
from groq import Groq
import gradio as gr
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import OpenAIEmbeddings

# Graph RAG
![title](images/graph_rag.png)

## Load model and prompts

In [2]:
FILENAME = 'data/un_regulations_157_graph.json'
USERNAME = 'neo4j'
URI = 'neo4j://localhost:7687'
MODEL_NAME = 'llama3-70b-8192'

NODE_SELECTION = """Goal:Identify the most relevant node.
Context:
The graph database contains a list of automotive regulation compliance text, each node represents a requirement or a definition.
Each node has a 'text' attribute which contains the actual regulation.
The relationships represent either a hierarchical relationship (node 3.2.1 is child_of 3.2) or a mention relation (node 4.2.5.1 mentions regulation 2.1.0)
Input:
- question: User's query
- list_of_nodes: A dict of relevant nodes w.r.t. user query, where the key is the node_id and the value node_text refers to the description of the node
Task: Your task is to identify the node thats most relevant for the input query.
Instructions:
- From the input nodes identify one node thats most relevant for the user's query (called n1) by analysing the relevance of the text to the query
- After having conducted the analysis, report only the end result and not the breakdown of the analysis itself.
- Return the id of the node in a yaml format with key 'output'
Note:
- Do not return anything else except the yaml output as the result shall be used by an automated code.
Example:
```
Question:
    "What are the procedures to take to ensure mechanical safety?"
List of nodes:
{{
    '4.3.1': "A transition demand shall not endanger the safety of the vehicle occupants or other road users.",
    '9.3.1': "Most guidelines relating to vehicle inspection and safety are referred to in 3.7.2",
    '3.5.4.2': "Procedures manual for a safe sotware updates are included below. This does not include mechanical activations"
}}
// it can be seen that '9.3.1' is the most relevant to the user's query
Output: '9.3.1'
```
OK, now here's the input
Question:
{query}
List_of_nodes:
{list_of_nodes}
Output:
"""

QA_PROMPT_TEMPLATE = """Task: Formulate an extensive answer to the question given the returned context and the user's query.
The context data contains a automotive compliance regulations, most of the time its either a requirement or a definition.
The regulations given in the context are identified by their id located at the beginning and followed by a colon (3.2.1.:, 2.4.: etc...). Include regulation id in your answer whenever necessary.
Use the following pieces of context to answer the query at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:
"""

QA_GENERATION_PROMPT = PromptTemplate(input_variables=["context", "question"], template=QA_PROMPT_TEMPLATE)

In [3]:
with open(FILENAME) as f:
    data = json.load(f)
all_nodes = [node["id"] for node in data]
graph = Neo4jGraph(url=URI, username=USERNAME, password=os.environ.get("NEO4J_PWD"))
llm_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1024,
    api_key=os.environ.get('OPEN_AI_KEY')
)

In [4]:
with open(FILENAME) as f:
    data = json.load(f)
all_nodes = [node["id"] for node in data]
graph = Neo4jGraph(url=URI, username=USERNAME, password=os.environ.get("NEO4J_PWD"))
llm_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1024,
    api_key=os.environ.get('OPEN_AI_KEY')
)
graph.refresh_schema()
print(graph.schema)

Node properties:
Node {id: STRING, text: STRING, embeddings: LIST}
Relationship properties:

The relationships:
(:Node)-[:child_of]->(:Node)
(:Node)-[:mentions]->(:Node)


## Helper functions

In [5]:
def get_similar_nodes(question, top_k):
    fetch_top_nodes="""WITH $query_embedding AS queryEmbedding
    MATCH (n:Node)
    WITH n, 
        reduce(dot = 0.0, i IN range(0, size(n.embeddings) - 1) | dot + n.embeddings[i] * queryEmbedding[i]) AS dot_product,
        reduce(norm1 = 0.0, i IN range(0, size(n.embeddings) - 1) | norm1 + n.embeddings[i] ^ 2) AS norm1,
        reduce(norm2 = 0.0, i IN range(0, size(queryEmbedding) - 1) | norm2 + queryEmbedding[i] ^ 2) AS norm2
    WITH n, dot_product / (sqrt(norm1) * sqrt(norm2)) AS similarity
    WHERE similarity > 0.0 // Optional: Filter negative similarities if desired
    ORDER BY similarity DESC
    RETURN n.id AS id, n.text as text, similarity as sim
    LIMIT $top_k;
    """
    query_embedding = embeddings_model.embed_query(question)
    similar_nodes = graph.query(fetch_top_nodes, {"query_embedding": query_embedding, "top_k":top_k})
    print(f'1. Retrieve data from vector store based on similarity:\nSimilar nodes to the query {[(result["id"], result["sim"]) for result in similar_nodes]}')
    return {result['id']: result['text'] for result in similar_nodes}

def get_most_relevant_node(question, similar_nodes_dict):
    query = NODE_SELECTION.format(query=question, list_of_nodes=similar_nodes_dict)
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        temperature=0.8,
        model=MODEL_NAME,
    )
    completion = chat_completion.choices[0].message.content
    most_relevant_node = ''
    list_tokens = completion.split(' ')
    for token in list_tokens:
        token = token.replace('"', '')
        token = token.replace("'", "")
        if token in all_nodes:
            most_relevant_node = token
            break
    if most_relevant_node == '':
        print('LLM failed to return a valid node')
        most_relevant_node = list(similar_nodes_dict.keys())[0]
    most_relevant_txt = similar_nodes_dict[most_relevant_node]
    print(f'2. LLM based decision on the most relevant nodeMost relevant node {most_relevant_node}: {most_relevant_txt}')
    return most_relevant_node, most_relevant_txt

def build_context(most_relevant_node, most_relevant_txt):
    fetch_related_nodes = f"""
    MATCH (n:Node {{id: '{most_relevant_node}'}})-[:mentions]->(mentionedNode:Node)
    RETURN mentionedNode.id AS id, mentionedNode.text AS text;
    """
    related_nodes = graph.query(fetch_related_nodes)
    print(f'3. Cypher query to generate the full context based on the most relevant node: {related_nodes}')

    # answer the query
    full_context = [most_relevant_node + ': ' + most_relevant_txt]
    for related_node in related_nodes:
        full_context.append(related_node["id"] + ': ' + related_node["text"])
    return full_context

def get_answer(question, context, max_new_tokens, temperature):
    query = QA_PROMPT_TEMPLATE.format(question=question, context=context)
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        temperature=temperature,
        model=MODEL_NAME,
        max_tokens=max_new_tokens
    )
    print(f'4. LLM based answer')
    return chat_completion.choices[0].message.content

def generate_qa(msg, max_new_tokens, temperature, top_k):
    similar_nodes = get_similar_nodes(msg, top_k)
    most_relevant_node, most_relevant_txt = get_most_relevant_node(msg, similar_nodes)
    full_context = build_context(most_relevant_node, most_relevant_txt)
    return get_answer(msg, full_context, max_new_tokens, temperature)

def generate_chat(msg, max_new_tokens, temperature, top_k, chat_history):
    similar_nodes = get_similar_nodes(msg, top_k)
    most_relevant_node, most_relevant_txt = get_most_relevant_node(msg, similar_nodes)
    full_context = build_context(most_relevant_node, most_relevant_txt)
    result = get_answer(msg, full_context, max_new_tokens, temperature)
    chat_history.append((msg, result))
    return "", chat_history

## QA interface

In [6]:
qa_interface = gr.Interface(fn=generate_qa, 
                            title="Graph RAG QA interface",
                            inputs=[gr.Textbox(label="Prompt"),
                                    gr.Slider(label="Max new tokens", value=200, maximum=1024, step=int, minimum=1),
                                    gr.Slider(label="Temperature", value=0.8, maximum=1.0, minimum=0.0),
                                    gr.Slider(label="Max documents retrieved", value=3, step=int, maximum=5, minimum=1)
                                   ], 
                            outputs=[gr.Textbox(label="Completion")]
                           )
gr.close_all()
qa_interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://b4fa58ad350676c04d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1. Retrieve data from vector store based on similarity:
Similar nodes to the query [('4.1.', 0.6936294134818086), ('3.1.', 0.6661224263630892), ('3.1.1.', 0.6470114174978703)]
2. LLM based decision on the most relevant nodeMost relevant node 4.1.:  If the vehicle type submitted for approval pursuant to this Regulation meets the requirements of paragraph 5 to 9 below, approval of that vehicle shall be granted. The functional operation of ‘The System’, as laid out in the documents required in paragraph 3, shall be tested as follows  Schematic system layout including sensors for the environmental perception (e.g. block diagram) … Parameters below are essential when describing the pattern of the traffic critical scenarios in section 2.1.
3. Cypher query to generate the full context based on the most relevant node: [{'id': '5.', 'text': ' System Safety and Fail-safe Response Test Specifications for ALKS'}, {'id': '9.', 'text': ' Cybersecurity and Software Updates'}]
4. LLM based answer


## Chatbot interface

In [29]:
with gr.Blocks() as chat_interface:
    title = gr.HTML("<center><h1>Graph RAG based retrieval</h1></center>")
    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(label="Prompt")
            max_new_tokens = gr.Slider(label="Max new tokens", value=200, maximum=1024, step=int, minimum=1)
            temperature = gr.Slider(label="Temperature", value=0.8, maximum=1.0, minimum=0.0)
            max_documents = gr.Slider(label="Max documents retrieved", value=3, step=int, maximum=5, minimum=1)
        with gr.Column():
            chatbot = gr.Chatbot(height=240)
    btn = gr.Button("Query")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
    btn.click(
        generate_chat,
        inputs=[
            msg,
            max_new_tokens,
            temperature,
            max_documents,
            chatbot],
        outputs=[msg, chatbot])
gr.close_all()
chat_interface.launch(share=True)

/home/moz9le/Documents/ml11/lib/python3.11/site-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f326dcb3e293f037e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1. Retrieve data from vector store based on similarity:
Similar nodes to the query [('4.1.', 0.6936294134818086), ('3.1.', 0.6661224263630892), ('3.1.1.', 0.6470114174978703)]
2. LLM based decision on the most relevant nodeMost relevant node 4.1.:  If the vehicle type submitted for approval pursuant to this Regulation meets the requirements of paragraph 5 to 9 below, approval of that vehicle shall be granted. The functional operation of ‘The System’, as laid out in the documents required in paragraph 3, shall be tested as follows  Schematic system layout including sensors for the environmental perception (e.g. block diagram) … Parameters below are essential when describing the pattern of the traffic critical scenarios in section 2.1.
3. Cypher query to generate the full context based on the most relevant node: [{'id': '5.', 'text': ' System Safety and Fail-safe Response Test Specifications for ALKS'}, {'id': '9.', 'text': ' Cybersecurity and Software Updates'}]
4. LLM based answer
1. R